## Solve using Benders

In [1]:
from gurobipy import *

In [2]:
import qminos
from qminos import qwarmLP

In [3]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [4]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

In [5]:
from cobra.io import load_json_model

ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

In [6]:
from dynamicme.optimize import Optimizer

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f1aa778ef80>)


In [7]:
opt = Optimizer(ijomc)

In [8]:
gap = opt.add_duality_gap_constraint()

In [9]:
crowding = ijomc.metabolites.crowding
crowdingfit = ijofit.metabolites.crowding
a12_dict = {(gap.metabolites.get_by_id(crowding.id), gap.reactions.get_by_id(rxn.id)):(
    rxn.metabolites[crowding], 
    ijofit.reactions.get_by_id(rxn.id).metabolites[crowdingfit]) for rxn in crowding.reactions}

In [10]:
opt.make_disjunctive_primal_dual(gap, a12_dict)

<Model duality_gap at 0x7f1aa724e910>

In [11]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary_'):
    rxn.objective_coefficient = 1.

In [12]:
ijofit.optimize()
MU_FIT = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

In [13]:
MU_FIT

0.7301837182736599

In [14]:
F_TOL = 0.05

gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = MU_FIT*(1+F_TOL)
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = MU_FIT*(1-F_TOL)

In [15]:
gap.optimize(solver='gurobi', objective_sense='minimize')

<Solution 1.00 at 0x7f1aa20ef0d0>

In [16]:
sum([r.x for r in gap.reactions.query('binary_')])

1.0

In [17]:
bvars = [r for r in gap.reactions if r.variable_kind == 'integer']
len(bvars)

2018

In [18]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7489994207811342

In [19]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        print rxn, rxn.x

binary_crowding_ATPS4rpp_abs 1.0


## Solve as MILP

In [20]:
from cobra.solvers import gurobi_solver
from gurobipy import *

In [21]:
%%time
milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

CPU times: user 299 ms, sys: 14.4 ms, total: 314 ms
Wall time: 297 ms


In [22]:
milp.Params.IntFeasTol = 1e-9
milp.Params.FeasibilityTol = 1e-8
milp.Params.OptimalityTol = 1e-8
milp.Params.Presolve = 0
milp.Params.NumericFocus = 1

In [23]:
milp.Params.OutputFlag=1
%time milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 22283 rows, 18524 columns and 79543 nonzeros
Variable types: 16506 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-11, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-02, 1e+03]
  RHS range        [5e-03, 1e+04]
Variable types: 16387 continuous, 2137 integer (2018 binary)

Root relaxation: objective 6.221914e-02, 16736 iterations, 1.74 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.06222    0    1          -    0.06222      -     -    1s
     0     0    0.08374    0    3          -    0.08374      -     -    2s
     0     0    0.23735    0    3          -    0.23735      -     -    2s
     0     2    0.23735    0    3          -    0.23735      -     -    2s
H  289    22                       1.0000000   

In [30]:
milp.Status == GRB.OPTIMAL

True

In [31]:
milp.ObjVal

1.0

In [29]:
sol = gurobi_solver.format_solution(milp, gap)

In [32]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        print rxn, rxn.x, xmilp

binary_crowding_ATPS4rpp_abs 1.0 1.0


## Solve using Benders with quad/dqq subproblem

In [40]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

decomposer = Decomposer(milp)
master,sub = decomposer.benders_decomp()
master._precision_sub = 'dqq'

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [41]:
master._verbosity = 1
master.optimize(cb_benders)

Optimize a model with 1 rows, 2019 columns and 2019 nonzeros
Variable types: 1 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Getting MINOS parameters...
Done in 487.401 seconds with status 0
Getting MINOS parameters...
Done in 1.8758 seconds with status 0
Getting MINOS parameters...
Done in 1.84436 seconds with status 0
Variable types: 1 continuous, 2018 integer (2018 binary)
Getting MINOS parameters...
Done in 1.86757 seconds with status 0

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.785988e-03   0.000000e+00    497s
       2    6.2219142e-02   0.000000e+00   0.000000e+00    497s

Root relaxation: objective 6.221914e-02, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [75]:
import numpy as np
yopt = np.array([y.X for y in decomposer._ys])
sum(yopt)

1.0

In [83]:
y_dict = {y.VarName:y.X for y in decomposer._ys}

In [78]:
from qminos.quadLP import QMINOS
### SOlve primal sub
qsolver = QMINOS()
sense_dict = {GRB.EQUAL:'E', GRB.LESS_EQUAL:'L', GRB.GREATER_EQUAL:'G'}
csenses = [sense_dict[c] for c in decomposer._csenses]
xp,statp,hsp = qsolver.solvelp(decomposer._A, decomposer._d - decomposer._B*yopt, decomposer._cx, decomposer._xl, decomposer._xu, csenses, precision='dqq')

Getting MINOS parameters...
Done in 25.9446 seconds with status 0


In [81]:
x_dict = {x.VarName:xp[j] for j,x in enumerate(decomposer._x0)}

In [96]:
milp.getVars()[7]

<gurobi.Var 7 (value 0.749027199175)>

In [91]:
x_dict['7']

0.7490271991752928

In [32]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        ybend = y_dict[]
        print rxn, rxn.x, xmilp

binary_crowding_ATPS4rpp_abs 1.0 1.0
